# Lab 5: Agent Memory 

Build agents with **long-term memory** using Azure AI Foundry's Memory API.

## What You'll Learn

| Scenario | Description |
|----------|-------------|
| **1. Memory Store** | Create stores with local models |
| **2. Store Memories** | Extract memories from conversations |
| **3. Scope Isolation** | Keep user data separate |
| **4. Agent + Memory** | Agent with `memory_search` tool |
| **5. Cross-Session** | Memory persists across sessions |

## Theme: Space Exploration Expert 🚀

This lab uses a **space exploration** theme - the agent remembers users' favorite planets, space interests, and exploration preferences.


## Prerequisites- `.env` file with `APIM_URL`, `APIM_KEY`, `MODEL_NAME`

- Complete **Lab 1A** (Landing Zone) - provides APIM gateway

## Step 1: Install Dependencies

In [ ]:
!pip install pandas requests azure-ai-projects==2.0.0b2 azure-identity openai -q

## Step 2: Load Landing Zone Configuration

In [ ]:
import os, subprocess, json
from pathlib import Path
from IPython.display import display, Markdown

# Load .env file
env_file = Path('/workspaces/getting-started-with-foundry/.env')
if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.strip() and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            os.environ[key] = value

# Landing zone config
APIM_URL = os.environ.get('APIM_URL', '')
APIM_KEY = os.environ.get('APIM_KEY', '')
GATEWAY_MODEL = os.environ.get('MODEL_NAME', 'gpt-4.1-mini')

print(f"✅ APIM URL: {APIM_URL[:50]}..." if APIM_URL else "❌ APIM_URL not set")
print(f"✅ APIM Key: {APIM_KEY[:8]}..." if APIM_KEY else "❌ APIM_KEY not set")
print(f"✅ Gateway Model: {GATEWAY_MODEL}")

## Step 3: Set Spoke Variables

In [4]:
# Spoke configuration
RG = "foundry-memory-spoke"
LOCATION = "eastus2"
LOCAL_CHAT_MODEL = "gpt-4.1-mini"
EMBEDDING_MODEL = "text-embedding-3-small"
MEMORY_STORE_NAME = "space-expert-memory"

PRINCIPAL_ID = subprocess.run(
    'az ad signed-in-user show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

display(Markdown(f'''
| Setting | Value |
|---------|-------|
| Resource Group | `{RG}` |
| Local Chat | `{LOCAL_CHAT_MODEL}` |
| Embedding | `{EMBEDDING_MODEL}` |
| Memory Store | `{MEMORY_STORE_NAME}` |
'''))


| Setting | Value |
|---------|-------|
| Resource Group | `foundry-memory-spoke` |
| Local Chat | `gpt-4.1-mini` |
| Embedding | `text-embedding-3-small` |
| Memory Store | `space-expert-memory` |


## Step 4: Create Resource Group

In [5]:
!az group create -n "{RG}" -l "{LOCATION}" -o table

Location    Name
----------  --------------------
eastus2     foundry-memory-spoke


## Step 4: Deploy Spoke Infrastructure

Deploys local models (for Memory API) + APIM connection. ⏱️ ~4-5 minutes

In [6]:
!az deployment group create -g "{RG}" --template-file spoke.bicep \
    -p deployerPrincipalId="{PRINCIPAL_ID}" \
    -p apimUrl="{APIM_URL}" \
    -p gatewayModelName="{GATEWAY_MODEL}" \
    -p localChatModel="{LOCAL_CHAT_MODEL}" \
    -p embeddingModelName="{EMBEDDING_MODEL}" \
    -p apimSubscriptionKey="{APIM_KEY}" \
    -o table

Name    State      Timestamp                         Mode         ResourceGroup
------  ---------  --------------------------------  -----------  --------------------
spoke   Succeeded  2026-01-27T16:20:45.152305+00:00  Incremental  foundry-memory-spoke


## Step 6: Get Deployment Outputs

In [ ]:
outputs = json.loads(subprocess.run(
    f'az deployment group show -g "{RG}" -n spoke --query properties.outputs -o json',
    shell=True, capture_output=True, text=True
).stdout)

ACCOUNT_NAME = outputs['accountName']['value']
PROJECT_NAME = outputs['projectName']['value']
PROJECT_ENDPOINT = outputs['projectEndpoint']['value']
LOCAL_CHAT = outputs['localChatModel']['value']
EMBEDDING = outputs['embeddingModelName']['value']

print(f"✅ Account: {ACCOUNT_NAME}")
print(f"✅ Project: {PROJECT_NAME}")
print(f"✅ Local Chat: {LOCAL_CHAT}")
print(f"✅ Embedding: {EMBEDDING}")

## Step 7: Wait for RBAC Propagation

In [8]:
import time
from IPython.display import clear_output

for i in range(60, 0, -10):
    clear_output(wait=True)
    print(f"⏳ RBAC propagation... {i}s")
    time.sleep(10)
clear_output(wait=True)
print("✅ Ready")

✅ Ready


## Step 8a: Setup Project Client

Use the SDK for clean Responses API access.

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
openai_client = project_client.get_openai_client()

print(f"✅ Project client ready: {PROJECT_ENDPOINT}")

## Step 8b: Setup Memory Client

In [10]:
from memory_helpers import MemoryClient, build_conversation
from display_helpers import show_store_created, show_memories, show_search_results, show_agent_created, show_conversation, show_error

memory = MemoryClient(ACCOUNT_NAME, PROJECT_NAME)
print(f"✅ Memory client ready")

✅ Memory client ready


---
# Scenario 1: Create Memory Store

The memory store uses **local models** for internal processing.

In [11]:
result = memory.create_store(
    name=MEMORY_STORE_NAME,
    chat_model=LOCAL_CHAT,
    embedding_model=EMBEDDING,
    description="Space exploration preferences and conversation history",
    user_profile_details="Favorite planets, space missions, exploration interests, celestial phenomena preferences"
)

if 'error' not in result:
    show_store_created(MEMORY_STORE_NAME, LOCAL_CHAT, EMBEDDING)
else:
    show_error(result['error'])

### Memory Store Created

Property,Value
Name,space-expert-memory
Chat Model,gpt-4.1-mini
Embedding Model,text-embedding-3-small
Status,✅ Created


---
# Scenario 2: Store User Memories

Extract and store memories from conversations using the Memory API.

In [12]:
# Test users with different space exploration profiles
USER_ALICE = "user_alice_123"
USER_BOB = "user_bob_456"

display(Markdown('''
| User | Scope ID | Profile |
|------|----------|---------|
| Alice | `user_alice_123` | Loves Mars, interested in rover missions, wants to see the northern lights |
| Bob | `user_bob_456` | Saturn fan, fascinated by rings and moons, dreams of Europa exploration |
'''))


| User | Scope ID | Profile |
|------|----------|---------|
| Alice | `user_alice_123` | Loves Mars, interested in rover missions, wants to see the northern lights |
| Bob | `user_bob_456` | Saturn fan, fascinated by rings and moons, dreams of Europa exploration |


In [13]:
# Store Alice's preferences
alice_msgs = build_conversation(
    "Mars is my absolute favorite planet! I'm fascinated by the Perseverance rover and Ingenuity helicopter missions. I also really want to see the northern lights on Earth someday - they're on my bucket list.",
    "Got it! Mars is your favorite, you love the rover missions, and you're dreaming of seeing the aurora borealis. I'll remember that!"
)

print("⏳ Processing Alice's memories...")
result = memory.update_memories(MEMORY_STORE_NAME, USER_ALICE, alice_msgs)

if 'error' not in result:
    show_memories("Alice's Memories Stored", result.get('memories', []))
else:
    show_error(result['error'])

⏳ Processing Alice's memories...
✅ Alice's Memories Stored - No new memories extracted


In [14]:
# Store Bob's preferences
bob_msgs = build_conversation(
    "Saturn is definitely my favorite - those rings are just spectacular! I'm really interested in its moon Europa and the possibility of life in its subsurface ocean. I also love following the James Webb telescope discoveries.",
    "Saturn fan with a love for those iconic rings! You're curious about Europa's ocean and following JWST discoveries. Got it!"
)

print("⏳ Processing Bob's memories...")
result = memory.update_memories(MEMORY_STORE_NAME, USER_BOB, bob_msgs)

if 'error' not in result:
    show_memories("Bob's Memories Stored", result.get('memories', []))
else:
    show_error(result['error'])

⏳ Processing Bob's memories...
✅ Bob's Memories Stored - No new memories extracted


---
# Scenario 3: Search Memories (Scope Isolation)

Verify each user only sees their own memories.

In [15]:
query = "Which planet should I learn more about?"
display(Markdown(f'**Query:** "{query}"'))

# Each user only sees their own memories
alice_result = memory.search_memories(MEMORY_STORE_NAME, USER_ALICE, query)
bob_result = memory.search_memories(MEMORY_STORE_NAME, USER_BOB, query)

show_search_results("Alice", "👩", alice_result.get('memories', []))
show_search_results("Bob", "👨", bob_result.get('memories', []))

display(Markdown('✅ **Scope isolation verified** - each user sees only their own memories'))

**Query:** "Which planet should I learn more about?"

#### 👩 Alice's Memories

Type,Content
user_profile,User's favorite planet is Mars.
user_profile,User has a bucket list goal to see the northern lights (aurora borealis) on Earth.
user_profile,"User is fascinated by Mars rover missions, especially Perseverance and Ingenuity."


#### 👨 Bob's Memories

Type,Content
user_profile,"User's favorite planet is Saturn, with a special appreciation for its rings."
user_profile,User is interested in Europa's subsurface ocean and the potential for extraterrestrial life there.
user_profile,User follows discoveries from the James Webb Space Telescope (JWST).


✅ **Scope isolation verified** - each user sees only their own memories

---
# Scenario 4: Agent with Memory

Create an agent that uses `memory_search` tool.

> ⚠️ **Current Limitation**: The `memory_search` tool is **not supported with BYO (gateway) models**.
> Error: `"The following tools are not supported with BYO model: memory_search. Please remove these tools or use a standard model deployment."`
> 
> **Workaround**: Use a local model deployment for agents with memory tools.
> Once this limitation is lifted, you can switch back to gateway models (`connection/model` format).

In [16]:
from azure.ai.projects.models import PromptAgentDefinition

AGENT_NAME = "SpaceExpert"

def create_agent_for_user(scope: str) -> tuple:
    """Create an agent scoped to a specific user."""
    agent = project_client.agents.create_version(
        agent_name=AGENT_NAME,
        definition=PromptAgentDefinition(
            model=LOCAL_CHAT,
            instructions="You are a friendly space exploration expert. Personalize recommendations based on user's favorite planets and space interests. Remember their specific interests in missions, phenomena, and celestial bodies. Always use the memory tool before giving an answer.",
            tools=[{
                "type": "memory_search",
                "memory_store_name": MEMORY_STORE_NAME,
                "scope": scope,
                "update_delay": 1
            }]
        )
    )
    return agent

# Create agents for each user
agent_alice = create_agent_for_user(USER_ALICE)
agent_bob = create_agent_for_user(USER_BOB)

display(Markdown('''
### Agents Created
| User | Agent Version | Memory Scope |
|------|--------------|--------------|
| Alice | `''' + agent_alice.version + '''` | `user_alice_123` |
| Bob | `''' + agent_bob.version + '''` | `user_bob_456` |

> ⚠️ Using local model (gateway not supported with `memory_search`)
'''))


### Agents Created
| User | Agent Version | Memory Scope |
|------|--------------|--------------|
| Alice | `16` | `user_alice_123` |
| Bob | `17` | `user_bob_456` |

> ⚠️ Using local model (gateway not supported with `memory_search`)


In [17]:
query = "Hi! I want to learn something fascinating about space today. What would you recommend based on my interests?"
display(Markdown(f'**Query:** "{query}"'))

# Alice's recommendation
response_alice = openai_client.responses.create(
    input=query,
    extra_body={"agent": {"name": agent_alice.name, "version": agent_alice.version, "type": "agent_reference"}}
)
alice_response = response_alice.output_text if hasattr(response_alice, 'output_text') else str(response_alice.output)

# Bob's recommendation
response_bob = openai_client.responses.create(
    input=query,
    extra_body={"agent": {"name": agent_bob.name, "version": agent_bob.version, "type": "agent_reference"}}
)
bob_response = response_bob.output_text if hasattr(response_bob, 'output_text') else str(response_bob.output)

display(Markdown('---'))
show_conversation("👩 Alice's Recommendation", query, alice_response)
display(Markdown('---'))
show_conversation("👨 Bob's Recommendation", query, bob_response)

display(Markdown('''
### ✅ Same query, different answers!
- **Alice** gets Mars/rover mission recommendations (loves Mars and Perseverance)
- **Bob** gets Saturn/Europa recommendations (fascinated by rings and subsurface oceans)
'''))

**Query:** "Hi! I want to learn something fascinating about space today. What would you recommend based on my interests?"

---

### 👩 Alice's Recommendation

Role,Message
👤 User,Hi! I want to learn something fascinating about space today. What would you recommend based on my interests?
🤖 Agent,"Since you love Mars and are fascinated by the Perseverance rover and Ingenuity helicopter, here’s something fascinating: Did you know that Perseverance carries a special instrument called MOXIE (Mars Oxygen In-Situ Resource Utilization Experiment)? MOXIE is designed to produce oxygen from the thin Martian atmosphere, which is mostly carbon dioxide. This is a groundbreaking step toward enabling future human missions on Mars, as it could allow astronauts to create breathable air and rocket fuel directly on the planet! Also, tying to your bucket list interest in the aurora borealis, Mars has its own version of auroras—but they are quite different from Earth’s. Mars’ auroras are more localized and appear in patches because Mars lacks a global magnetic field like Earth does. These Martian auroras glow in ultraviolet light and were discovered by the MAVEN spacecraft studying the Martian atmosphere. Would you like to hear more about the latest findings from Perseverance or how Ingenuity’s flights are helping us explore Mars? Or maybe more about Martian auroras?"


---

### 👨 Bob's Recommendation

Role,Message
👤 User,Hi! I want to learn something fascinating about space today. What would you recommend based on my interests?
🤖 Agent,"Since you love Saturn and its iconic rings, and have a keen interest in Europa's subsurface ocean and the possibility of life there, plus you follow discoveries from the James Webb Space Telescope (JWST), I have a fascinating space tidbit for you: Recently, JWST has been providing incredibly detailed infrared observations that can help scientists understand the composition of icy moons and gas giants. While Europa is a moon of Jupiter (not Saturn), JWST's spectroscopic capabilities allow us to study the surface ices and possible plumes on Europa, searching for organic molecules or signs of habitability hidden beneath its icy shell. Additionally, Saturn’s rings themselves continue to surprise us. JWST’s infrared observations help scientists analyze the composition of Saturn's rings, revealing how their particles might age and interact with Saturn’s magnetosphere. This gives clues about the rings’ origins and their dynamic changes over time. Would you like me to share the latest findings from JWST about Saturn’s rings or recent intriguing studies related to Europa’s ocean and potential biosignatures?"



### ✅ Same query, different answers!
- **Alice** gets Mars/rover mission recommendations (loves Mars and Perseverance)
- **Bob** gets Saturn/Europa recommendations (fascinated by rings and subsurface oceans)


---
# Scenario 5: Automatic Memory Extraction

Demonstrate that the agent **automatically learns** from conversations - no manual `update_memories()` needed!

> 📝 **How it works:**
> - The `memory_search` tool has `update_delay` set (we use 1 second for demo)
> - After each response, the system automatically extracts memories
> - Chat summaries are enabled in our memory store (`chat_summary_enabled: True`)

In [18]:
USER_CHARLIE = "user_charlie_789"
agent_charlie = create_agent_for_user(USER_CHARLIE)

In [19]:
display(Markdown('### Turn 1: Charlie chats with the agent'))

charlie_msg1 = "Hi! I'm really excited about the upcoming solar eclipse next month. I want to find the best viewing spot and learn about what causes them."

response1 = openai_client.responses.create(
    input=charlie_msg1,
    extra_body={"agent": {"name": agent_charlie.name, "version": agent_charlie.version, "type": "agent_reference"}}
)
charlie_response1 = response1.output_text if hasattr(response1, 'output_text') else str(response1.output)

show_conversation("Charlie's First Message", charlie_msg1, charlie_response1, "Charlie")

# Continue the conversation
charlie_msg2 = "That sounds great! By the way, Jupiter is my favorite planet. I love learning about the Great Red Spot and the Galilean moons."

response2 = openai_client.responses.create(
    input=charlie_msg2,
    extra_body={"agent": {"name": agent_charlie.name, "version": agent_charlie.version, "type": "agent_reference"}}
)
charlie_response2 = response2.output_text if hasattr(response2, 'output_text') else str(response2.output)

display(Markdown('---'))
show_conversation("Charlie's Second Message", charlie_msg2, charlie_response2, "Charlie")

### Turn 1: Charlie chats with the agent

### Charlie's First Message

Role,Message
👤 Charlie,Hi! I'm really excited about the upcoming solar eclipse next month. I want to find the best viewing spot and learn about what causes them.
🤖 Agent,"That's awesome! Solar eclipses are incredible celestial events. They occur when the Moon passes between the Earth and the Sun, temporarily blocking out the Sun's light either partially or completely for viewers in the path. For the best viewing spot, it's important to be within the path of totality where the total eclipse is visible. The exact path changes with each eclipse. If you tell me your location or where you're willing to travel, I can help identify the most optimal viewing spots for the upcoming eclipse next month. Also, make sure to use proper eye protection like eclipse glasses to safely enjoy the event! Would you like me to recommend some safe viewing gear too?"


---

### Charlie's Second Message

Role,Message
👤 Charlie,"That sounds great! By the way, Jupiter is my favorite planet. I love learning about the Great Red Spot and the Galilean moons."
🤖 Agent,"Jupiter is an incredible favorite! The Great Red Spot is such a fascinating storm—it's been raging for centuries and is so massive that it could swallow Earth whole. The Galilean moons—Io, Europa, Ganymede, and Callisto—are equally amazing, each with unique characteristics like volcanic activity on Io and the potential subsurface ocean on Europa. If you're interested, I can share updates on missions studying Jupiter and its moons, like NASA's Juno probe or the upcoming Europa Clipper mission focused on Europa's habitability. Would you like to know more about these missions or some intriguing phenomena on Jupiter and its moons?"


In [20]:
# Wait for automatic memory extraction
display(Markdown('### ⏳ Waiting for automatic memory extraction...'))
display(Markdown('> The `memory_search` tool automatically extracts and stores memories after `update_delay` seconds of inactivity.'))

import time
for i in range(30, 0, -10):
    clear_output(wait=True)
    display(Markdown(f'### ⏳ Waiting for memory extraction... {i}s'))
    time.sleep(10)

clear_output(wait=True)
print("✅ Memory extraction should be complete")

✅ Memory extraction should be complete


In [21]:
# Turn 2: New conversation - ask what we talked about
display(Markdown('### Turn 2: New conversation - test if agent remembers'))

charlie_msg3 = "What have we recently been talking about?"

response3 = openai_client.responses.create(
    input=charlie_msg3,
    extra_body={"agent": {"name": agent_charlie.name, "version": agent_charlie.version, "type": "agent_reference"}}
)
charlie_response3 = response3.output_text if hasattr(response3, 'output_text') else str(response3.output)

show_conversation("Charlie Asks About Previous Chat", charlie_msg3, charlie_response3, "Charlie")

display(Markdown('''
### ✅ Automatic Memory Extraction Works!

**What just happened:**
1. Charlie chatted about solar eclipses & Jupiter being their favorite planet
2. We did **NOT** call `update_memories()` manually
3. The `memory_search` tool automatically extracted and stored the conversation
4. In a new conversation, the agent remembers what we discussed!

**This is the magic of the `memory_search` tool** - it handles extraction automatically.
'''))

### Turn 2: New conversation - test if agent remembers

### Charlie Asks About Previous Chat

Role,Message
👤 Charlie,What have we recently been talking about?
🤖 Agent,"Hey! We've recently been chatting about your excitement for the upcoming solar eclipse in February 2026. You were curious about the causes of solar eclipses—how the Moon blocks the Sun's light when it passes between Earth and the Sun, creating the eclipse for those in the path of totality. We also talked about finding the best spots to view this amazing event and the importance of using proper eye protection. Alongside that, you've shared your love for Jupiter, especially the Great Red Spot and its fascinating Galilean moons—I’ve told you about the storm’s massive size and longevity, and unique features of moons like Io and Europa. If you want, I can keep you updated on missions like Juno and Europa Clipper too! Would you like me to help you pick some ideal places to watch the eclipse or dive back into Jupiter’s wonders?"



### ✅ Automatic Memory Extraction Works!

**What just happened:**
1. Charlie chatted about solar eclipses & Jupiter being their favorite planet
2. We did **NOT** call `update_memories()` manually
3. The `memory_search` tool automatically extracted and stored the conversation
4. In a new conversation, the agent remembers what we discussed!

**This is the magic of the `memory_search` tool** - it handles extraction automatically.


---
# Summary

## Key Learnings

| Concept | Detail |
|---------|--------|
| Memory API Models | Must be deployed **locally** (not via gateway) |
| Agent with `memory_search` | Also requires local model |
| Token Audience | `https://ai.azure.com` |
| Responses API | `openai_client.responses.create()` with `agent_reference` |

## Current Limitation

> ⚠️ **`memory_search` tool does not support BYO (gateway) models**
> 
> Error: `"The following tools are not supported with BYO model: memory_search"`

## Files

| File | Purpose |
|------|---------|
| `memory_helpers.py` | `MemoryClient` class, `build_conversation()` |
| `display_helpers.py` | Display functions for tables and results |
| `spoke.bicep` | Infrastructure (local models + APIM connection) |

In [22]:
# Uncomment to delete all resources
# !az group delete -n "{RG}" --yes --no-wait
# print(f"🗑️ Deleting resource group: {RG}")